Week 3: Toronto Neighbourhood Segementation and Clustering

In [3]:
#Importing necessary packages
import requests
import pandas as pd
import numpy as np

#Scraping the webpage using requests
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df_list = pd.read_html(html,header =0, flavor = 'bs4')
df = df_list[0]
print("Initial shape of data frame",df.shape)

Initial shape of data frame (287, 3)


Cleansing the dataset as per requirement

In [4]:
df=df[df['Borough'] != 'Not assigned']
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()


Grouping data throught borough and neighbourhood

In [9]:
df['Neighbourhood']=np.where(df['Neighbourhood']=='Not assigned', df['Borough'], df['Neighbourhood'])
print("Shape of dataframe after cleaning and grouping",df.shape)


Shape of dataframe after cleaning and grouping (103, 3)


Grouping data using geospatial file 

In [13]:
df2=pd.read_csv("https://cocl.us/Geospatial_data")
df2.columns=["Postcode","Latitude","Longitude"]
df_join=pd.merge(df,df2,on=['Postcode'],how='inner')

In [17]:
df3=df_join[['Postcode','Borough','Neighbourhood','Latitude','Longitude']].copy()
print(df3.head(5))

  Postcode      Borough                           Neighbourhood   Latitude  \
0      M1B  Scarborough                          Rouge, Malvern  43.806686   
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3      M1G  Scarborough                                  Woburn  43.770992   
4      M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  


The number of borough and neighbourhood after geospatial grouping 

In [24]:
print("This dataframe has {} borough and {} neighbourhoods".format(len(df3['Borough'].unique()),df3.shape[0]))

This dataframe has 11 borough and 103 neighbourhoods


Exploring neighbours using visualization with maps

In [27]:
#Importing necessary packages
import json
from geopy.geocoders import Nominatim

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



In [30]:
geo_loc = Nominatim()
loc = geo_loc.geocode('Toronto, Canada')
lat = loc.latitude
long = loc.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(lat, long))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [35]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[lat, long], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
toronto_map